In [2]:
#resize
def resize_vid(fname, vid_folder):
    
    try:


        from pathlib import Path

        re_dir = os.path.join(os.path.abspath(vid_folder), "proc_vids\(copy\)")

        Path(re_dir).mkdir(parents=True, exist_ok=True)

        re_fname = os.path.join(os.path.abspath(re_dir), fname)

        fname_abs = os.path.join(os.path.abspath(vid_folder), fname)

        #print("ffmpeg -i {} -vf scale=98:-2 {}".format(fname_abs, re_fname))
        os.system("ffmpeg -i {} -vf scale=96:96 {}".format(fname_abs, re_fname))
    except:
        print("error", fname)
    
    return ""



In [3]:
import os, sys, time, math, torch, pandas as pd, numpy as np, glob, skvideo.io

from torch import nn, optim
from torch.autograd import Variable
from pathlib import Path

sys.path.insert(0,"../")


from src.mocogan_model import Discriminator_I, Discriminator_V, Generator_I, GRU

from importlib import reload
import src
reload(src.mocogan_model)

from src.mocogan_model import Discriminator_I, Discriminator_V, Generator_I, GRU

In [4]:
cuda       = -1#cpu
ngpu       = 1
batch_size = 16
n_iter     = 120000
pre_train  = False

In [5]:
seed = 0
torch.manual_seed(seed)
np.random.seed(seed)
if cuda == True:
    torch.cuda.set_device(0)


In [6]:
DATA_DIR = os.path.abspath(os.path.join("..","data"))

VID_DIR = os.path.join(DATA_DIR, 'raw', "UCF101")

PROC_VID_DIR = os.path.join(VID_DIR, 'proc_vids(copy)')

MODEL_DIR = os.path.join(DATA_DIR, 'models')

OUTPUT_DIR = os.path.join(DATA_DIR, 'outputs')

In [7]:
def read_vid(fname, dir_name):
    
    abs_fname = os.path.join(dir_name,fname)
    
    #print(fname)
    
    return skvideo.io.vread(abs_fname)

In [8]:
files = os.listdir(PROC_VID_DIR)

videos = [ read_vid(file, PROC_VID_DIR) for file in files[:10] ]

# transpose each video to (nc, n_frames, img_size, img_size), and divide by 255
videos = [ video.transpose(3, 0, 1, 2) / 255.0 for video in videos ]

In [9]:
n_videos = len(videos)

T = 16

In [10]:
# for true video
def trim(video):
    start = np.random.randint(0, video.shape[1] - (T+1))
    end = start + T
    #print(start, end)
    return video[:, start:end, :, :]

def trim_noise(noise):
    start = np.random.randint(0, noise.size(1) - (T+1))
    end = start + T
    return noise[:, start:end, :, :, :]

def random_choice():
    X = []
    for _ in range(batch_size):
        video = videos[np.random.randint(0, n_videos-1)]
        video = torch.Tensor(trim(video))
        X.append(video)
    #print(X.shape)
    X = torch.stack(X)
    return X

In [11]:
# video length distribution
video_lengths = [video.shape[1] for video in videos]

In [12]:
''' set models '''

img_size = 96

nc = 3

ndf = 64 # from dcgan

ngf = 64

d_E = 10

hidden_size = 100 # guess

d_C = 50

d_M = d_E

nz  = d_C + d_M

criterion = nn.BCELoss()

In [13]:
dis_i = Discriminator_I(nc, ndf, ngpu=ngpu)

dis_v = Discriminator_V(nc, ndf, T=T, ngpu=ngpu)

gen_i = Generator_I(nc, ngf, nz, ngpu=ngpu)

gru = GRU(d_E, hidden_size, gpu=cuda)

gru.initWeight()

../src/mocogan_model.py:145: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  init.xavier_uniform(params)
../src/mocogan_model.py:155: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  init.constant(params, 0)


In [25]:
print(sum(p.numel() for p in dis_v.parameters() if p.requires_grad))

print(sum(p.numel() for p in dis_i.parameters() if p.requires_grad))

print(sum(p.numel() for p in gen_i.parameters() if p.requires_grad))

print(sum(p.numel() for p in gru.parameters() if p.requires_grad))

11042561
2775808
3863424
34610


In [13]:
''' prepare for train '''

label = torch.FloatTensor()

def timeSince(since):
    now = time.time()
    s = now - since
    d = math.floor(s / ((60**2)*24))
    h = math.floor(s / (60**2)) - d*24
    m = math.floor(s / 60) - h*60 - d*24*60
    s = s - m*60 - h*(60**2) - d*24*(60**2)
    return '%dd %dh %dm %ds' % (d, h, m, s)



def checkpoint(model, optimizer, epoch):
    
    filename = os.path.join(MODEL_DIR, '%s_epoch-%d' % (model.__class__.__name__, epoch))
    
    torch.save(model.state_dict(), filename + '.model')
    
    torch.save(optimizer.state_dict(), filename + '.state')

def save_video(fake_video, epoch):
    
    outputdata = fake_video * 255
    
    outputdata = outputdata.astype(np.uint8)
    
    gen_vid_dir = os.path.join(OUTPUT_DIR, 'generated_videos')
    
    Path(gen_vid_dir).mkdir(parents=True, exist_ok=True)
    
    file_path = os.path.join(gen_vid_dir, 'fakeVideo_epoch-%d.mp4' % epoch)
    
    skvideo.io.vwrite(file_path, outputdata)


In [14]:
''' adjust to cuda '''

if cuda == True:
    
    dis_i.cuda()
    
    dis_v.cuda()
    
    gen_i.cuda()
    
    gru.cuda()
    
    criterion.cuda()
    
    label = label.cuda()

In [15]:
# setup optimizer
lr = 0.0002

betas=(0.5, 0.999)

optim_Di  = optim.Adam(dis_i.parameters(), lr=lr, betas=betas)

optim_Dv  = optim.Adam(dis_v.parameters(), lr=lr, betas=betas)

optim_Gi  = optim.Adam(gen_i.parameters(), lr=lr, betas=betas)

optim_GRU = optim.Adam(gru.parameters(),   lr=lr, betas=betas)

In [16]:
''' use pre-trained models '''

if pre_train == True:
    dis_i.load_state_dict(torch.load(trained_path + '/Discriminator_I.model'))
    dis_v.load_state_dict(torch.load(trained_path + '/Discriminator_V.model'))
    gen_i.load_state_dict(torch.load(trained_path + '/Generator_I.model'))
    gru.load_state_dict(torch.load(trained_path + '/GRU.model'))
    optim_Di.load_state_dict(torch.load(trained_path + '/Discriminator_I.state'))
    optim_Dv.load_state_dict(torch.load(trained_path + '/Discriminator_V.state'))
    optim_Gi.load_state_dict(torch.load(trained_path + '/Generator_I.state'))
    optim_GRU.load_state_dict(torch.load(trained_path + '/GRU.state'))



In [23]:
''' calc grad of models '''

def bp_i(inputs, y, retain=False):
    
    label.resize_(inputs.size(0)).fill_(y)
    
    labelv = Variable(label)
    
    outputs = dis_i(inputs)
    
    err = criterion(outputs, labelv)
    
    err.backward(retain_graph=retain)
    
    return err.data, outputs.data.mean()

def bp_v(inputs, y, retain=False):
    
    label.resize_(inputs.size(0)).fill_(y)
    
    labelv = Variable(label)
    
    outputs = dis_v(inputs)
    
    err = criterion(outputs, labelv)
    
    err.backward(retain_graph=retain)
    
    return err.data, outputs.data.mean()


In [24]:
''' gen input noise for fake video '''

def gen_z(n_frames):
    
    z_C = Variable(torch.randn(batch_size, d_C))
    
    #  repeat z_C to (batch_size, n_frames, d_C)
    
    z_C = z_C.unsqueeze(1).repeat(1, n_frames, 1)
    
    #print(z_C.shape)
    
    eps = Variable(torch.randn(batch_size, d_E))
    
    #print(eps.shape)
    
    if cuda == True:
        z_C, eps = z_C.cuda(), eps.cuda()

    gru.initHidden(batch_size)
    
    # notice that 1st dim of gru outputs is seq_len, 2nd is batch_size
    
    z_M = gru(eps, n_frames).transpose(1, 0)
    
    #print(z_M.shape)
    
    z = torch.cat((z_M, z_C), 2)  # z.size() => (batch_size, n_frames, nz)
    
    #print(z.shape)
    
    return z.view(batch_size, n_frames, nz, 1, 1)


In [25]:
s = trim_noise(gen_z(18))

s.contiguous().view(batch_size*T, nz, 1, 1).shape

torch.Size([256, 60, 1, 1])

In [1]:
dis_v

NameError: name 'dis_v' is not defined

In [27]:
''' train models '''

start_time = time.time()

for epoch in range(1, n_iter+1):
    
    ''' prepare real images '''
    # real_videos.size() => (batch_size, nc, T, img_size, img_size)
    
    real_videos = random_choice()
    
    if cuda == True:
        
        real_videos = real_videos.cuda()
        
    real_videos = Variable(real_videos)
    
    real_img = real_videos[:, :, np.random.randint(0, T), :, :]

    ''' prepare fake images '''
    # note that n_frames is sampled from video length distribution
    
    n_frames = video_lengths[np.random.randint(0, n_videos)]
    
    Z = gen_z(n_frames)  # Z.size() => (batch_size, n_frames, nz, 1, 1)
    
    # trim => (batch_size, T, nz, 1, 1)
    
    Z = trim_noise(Z)
    # generate videos
    
    Z = Z.contiguous().view(batch_size*T, nz, 1, 1)
    # concatenate samples and frames into samples*frame samples of vectors.
    #Each vector will now have a dim of (d_M+d_C)*1*1
    
    fake_videos = gen_i(Z)
    
    fake_videos = fake_videos.view(batch_size, T, nc, img_size, img_size)
    
    # transpose => (batch_size, nc, T, img_size, img_size)
    
    fake_videos = fake_videos.transpose(2, 1)
    
    # img sampling
    fake_img = fake_videos[:, :, np.random.randint(0, T), :, :]

    ''' train discriminators '''
    # video
    dis_v.zero_grad()
    
    err_Dv_real, Dv_real_mean = bp_v(real_videos, 0.9)
    
    err_Dv_fake, Dv_fake_mean = bp_v(fake_videos.detach(), 0)
    
    err_Dv = err_Dv_real + err_Dv_fake
    
    optim_Dv.step()
    
    # image
    dis_i.zero_grad()
    
    err_Di_real, Di_real_mean = bp_i(real_img, 0.9)
    
    err_Di_fake, Di_fake_mean = bp_i(fake_img.detach(), 0)
    
    err_Di = err_Di_real + err_Di_fake
    
    optim_Di.step()


    ''' train generators '''
    
    gen_i.zero_grad()
    
    gru.zero_grad()
    
    # video. notice retain=True for back prop twice
    err_Gv, _ = bp_v(fake_videos, 0.9, retain=True)
    # images
    err_Gi, _ = bp_i(fake_img, 0.9)
    
    optim_Gi.step()
    
    optim_GRU.step()

    if epoch % 5 == 0:
        
        print('[%d%d] (%s) Loss_Di: %.4f Loss_Dv: %.4f Loss_Gi: %.4f Loss_Gv: %.4f Di_real_mean %.4f Di_fake_mean %.4f Dv_real_mean %.4f Dv_fake_mean %.4f'
              % (epoch, n_iter, timeSince(start_time), err_Di, err_Dv, err_Gi, err_Gv, Di_real_mean, Di_fake_mean, Dv_real_mean, Dv_fake_mean))

    if epoch % 10 == 0:
        
        save_video(fake_videos[0].data.cpu().numpy().transpose(1, 2, 3, 0), epoch)

    if epoch % 10 == 0:
        checkpoint(dis_i, optim_Di, epoch)
        checkpoint(dis_v, optim_Dv, epoch)
        checkpoint(gen_i, optim_Gi, epoch)
        checkpoint(gru,   optim_GRU, epoch)

0 16
25 41
42 58
28 44
68 84
122 138
135 151
61 77
5 21
8 24
463 479
139 155
39 55
45 61
74 90
177 193
51 67
58 74
55 71
173 189
65 81
53 69
94 110
80 96
124 140
58 74
73 89
13 29
38 54
136 152
135 151
22 38
209 225
468 484
66 82
144 160
114 130
24 40
100 116
87 103
54 70
20 36
101 117
151 167
29 45
115 131
119 135
86 102
83 99
148 164
42 58
20 36
68 84
47 63
43 59
31 47
9 25
91 107
83 99
274 290
43 59
12 28
1 17
39 55
134 150
98 114
28 44
26 42
248 264
135 151
72 88
135 151
21 37
108 124
38 54
71 87
0 16
91 107
7 23
101 117
[5http://localhost:8888/notebooks/notebooks/ucf101_mocogan.ipynb#/120000] (0d 0h 2m 37s) Loss_Di: 0.4000 Loss_Dv: 0.4405 Loss_Gi: 5.7454 Loss_Gv: 10.8776 Di_real_mean 0.8719 Di_fake_mean 0.0339 Dv_real_mean 0.9767 Dv_fake_mean 0.0000
104 120
38 54
74 90
123 139
26 42
32 48
97 113
59 75
178 194
132 148
93 109
145 161
148 164
119 135
105 121
4 20
35 51
90 106
26 42
98 114
51 67
208 224
188 204
104 120
10 26
141 157
94 110
173 189
15 31
166 182
171 187
7 23
78 94
100 

KeyboardInterrupt: 

In [22]:
%debug

> <ipython-input-17-c2de0ea9a299>(29)bp_v()
     25     err = criterion(outputs, labelv)
     26 
     27     err.backward(retain_graph=retain)
     28 
---> 29     return err.data[0], outputs.data.mean()

ipdb> err.data
tensor(0.6603)
ipdb> err.data[0]
*** IndexError: invalid index of a 0-dim tensor. Use tensor.item() to convert a 0-dim tensor to a Python number
ipdb> outputs.data
tensor([0.5278, 0.6354, 0.6683, 0.5604, 0.5503, 0.5802, 0.5894, 0.4852, 0.5013,
        0.4791, 0.4914, 0.4683, 0.3157, 0.4077, 0.6962, 0.5626])
ipdb> outputs.data.mean()
tensor(0.5325)
ipdb> s


In [163]:
s = trim_noise(gen_z(18))

In [136]:
nz

60